In [39]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.keys import Keys
import datetime
import time
import pandas as pd
import chromedriver_binary   

def move_textbox(driver, id, atai,wait):
    start = time.time()
    while driver.find_element_by_id(id).get_attribute('value')!=atai:
        driver.find_element_by_id(id).clear()
        wait.until(EC.visibility_of_element_located((By.ID, id)))        
        driver.find_element_by_id(id).send_keys(atai)
        driver.find_element_by_id(id).send_keys(Keys.ENTER)
        if time.time()-start > 5 :
            return False
    return True
def gs_scraper(search_place, driver_PATH, wait_time):
    options = Options()
    options.add_argument('--headless')
    driver = webdriver.Chrome(options=options)
    driver.get('https://gogo.gs/search/')
    wait = WebDriverWait(driver, wait_time)
    move_textbox(driver,"kw", search_place, wait)
    search_num = int((driver.find_element_by_xpath('/html/body/div[1]/div/div[1]/div[3]/div[1]').text)[7:])
    master_df = pd.DataFrame()
    while master_df.shape[0] < search_num :
        class_gs_list = driver.find_elements_by_class_name('card.search-shop')
        i = 0
        while (master_df.shape[0] < search_num) and (i<len(class_gs_list)): ## dataが十分かと繰り返しの回数を制限
            count = 0
            temp_df = pd.DataFrame()
            detail_df = pd.DataFrame()
            for cost in class_gs_list[i].find_elements_by_class_name('table.table-bordered'):
                count += 1 ##会員価格をみるときは名前などの情報はみんでもええから何回目かカウント(想定では1以上はないはず)
                regular_cost = cost.find_element_by_xpath('//tr[2]/td[2]')
                haioku_cost = cost.find_element_by_xpath('//tr[3]/td[2]')
                keiyu_cost = cost.find_element_by_xpath('//tr[4]/td[2]')
                if count == 1 :
                    #detail_name_list = class_gs_list[i].find_elements_by_class_name('shop-name.font-weight-bold')
                    detail_info_list = class_gs_list[i].find_elements_by_class_name('service-icons.mt-3 my-2')
                    #detail_name_list_tx = []
                    detail_info_list_tx = []
                    #for detail_n in detail_name_list:
                    #    detail_name_list_tx.append(detail_n.text[:-1])
                    for detail_i in detail_info_list:
                        detail_info_list_tx.append(detail_i.text)
                    #detail_df = pd.DataFrame(detail_info_list_tx,index=detail_name_list_tx).T
                    if (class_gs_list[i].find_element_by_class_name('shop-title').find_element_by_tag_name("img").get_attribute("src")) == "https://d1siwbe4ewvpee.cloudfront.net/maker/maker_8_96x96.png" :
                        brand_name = "出光"
                    elif (class_gs_list[i].find_element_by_class_name('shop-title').find_element_by_tag_name("img").get_attribute("src")) == "https://d1siwbe4ewvpee.cloudfront.net/maker/maker_3_96x96.png" :
                        brand_name = "ENEOS"
                    elif (class_gs_list[i].find_element_by_class_name('shop-title').find_element_by_tag_name("img").get_attribute("src")) == "https://d1siwbe4ewvpee.cloudfront.net/maker/maker_6_96x96.png" :
                        brand_name = "コスモ石油"
                    elif (class_gs_list[i].find_element_by_class_name('shop-title').find_element_by_tag_name("img").get_attribute("src")) == "https://d1siwbe4ewvpee.cloudfront.net/ext_maker/ext_maker_14_96x96.png" :
                        brand_name = "丸紅エネルギー"
                    elif (class_gs_list[i].find_element_by_class_name('shop-title').find_element_by_tag_name("img").get_attribute("src")) == "https://d1siwbe4ewvpee.cloudfront.net/maker/maker_7_96x96.png" :
                        brand_name = "昭和シェル"
                    elif (class_gs_list[i].find_element_by_class_name('shop-title').find_element_by_tag_name("img").get_attribute("src")) == "https://d1siwbe4ewvpee.cloudfront.net/maker/maker_99_96x96.png" :
                        brand_name = "その他"
                    else :
                        brand_name = "nan"
                    temp_df['name'] = [class_gs_list[i].find_element_by_class_name('shop-name.font-weight-bold').text]
                    temp_df['address'] = [class_gs_list[i].find_element_by_class_name('shop-address').text]
                    temp_df['brand'] = [brand_name]
                    temp_df["open_time"] = [class_gs_list[i].find_element_by_class_name('open-time').text]
                    temp_df['cash_r'] = [regular_cost.text]
                    temp_df['cash_h'] = [haioku_cost.text]
                    temp_df['cash_k'] = [keiyu_cost.text]
                    #temp_df['distance'] = [class_gs_list[i].find_element_by_class_name('list_item_distance_frame').text]
                else:
                    temp_df['mem_r'] = [regular_cost.text]
                    temp_df['mem_h'] = [haioku_cost.text]
                    temp_df['mem_k'] = [keiyu_cost.text]
            i += 1
            temp_df = pd.concat([detail_df,temp_df],axis=1)
            master_df = master_df.append(temp_df)
        #指定したボタンが表示されクリック出来る状態になるまで待機する
        if master_df.shape[0] < search_num :
            try:
                wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@rel="next"]')))
                next_button = driver.find_element_by_xpath('//*[@rel="next"]')
                next_button.click()
            except:
                print("Time Out Error!")
                break
    master_df = master_df.reset_index(drop=True)
    driver.quit()
    return master_df

In [40]:
#引数
search_place = "京都府京都市"
driver_PATH = "User/yamadakoki/yamada/school/tue3-5/group2app/chromedriver.exe"
wait_time = 10   #ボタン押しの実行待ち時間
#実行
data = gs_scraper(search_place, driver_PATH, wait_time)
display(data)

,name,address,brand,open_time,cash_r,cash_h,cash_k,mem_r,mem_h,mem_k
0,セルフMK山科ボウル前SS / エムケイ石油(株),京都府京都市山科区西野楳本町108-3,出光,24時間営業,151,162,131,151,162,131
1,best value station竹田SS / エムケイ石油(株),京都府京都市伏見区竹田中川原町40,その他,24時間営業,151,162,131,151,162,131
2,best value station御室SS / エムケイ石油(株),京都府京都市右京区宇多野馬場町29-1,その他,営業時間 07:00 - 23:00,151,162,131,151,162,131
3,セルフMK醍醐小野SS / エムケイ石油(株),京都府京都市山科区小野西浦47,出光,営業時間 07:00 - 21:00,151,162,131,151,162,131
4,鳥羽SS / 吉田商事(株),京都府京都市伏見区下鳥羽広長町155,出光,営業時間 07:00 - 21:00,151,162,131,151,162,131
...,...,...,...,...,...,...,...,...,...,...
141,淀SS / (有)伏見屋商店,京都府京都市伏見区淀際目町329,ENEOS,営業時間 07:30 - 20:30,-,-,-,-,-,-
142,五条七本松SS / トーア油業(株),京都府京都市下京区中堂寺北町42-3,ENEOS,営業時間 07:30 - 20:00,-,-,-,-,-,-
143,周山SS / 北桑石油(株),京都府京都市右京区京北周山町泓18-1,コスモ石油,営業時間 07:30 - 19:30,-,-,-,-,-,-
144,京北SS / 京北石油(株),京都府京都市右京区京北下中町下河原1,コスモ石油,営業時間,-,-,-,-,-,-


In [42]:
data.to_csv('./gasoline.csv')